In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )

/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
# unit-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

     
for seed in tqdm(range(repeat_num)):
    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_nodes,))   
    
    for num_step in [0]: 
        print(f"Seed: {seed}")           
        torch.manual_seed(2)
        p_list = ramps[num_step:]         
        g_feat_list = []
        y_list = []
                
        for p in p_list:            
            z = (rollout_index < np.quantile(rollout_index, p))
            
            nx.set_node_attributes(g, 0, "z")
            nx.set_node_attributes(g, {unit:1 for unit in range(num_nodes) if z[unit]}, "z")
            po_linear_model(g, gamma = 1)
            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
            
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/2step_CR_unit.pkl")        


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.02, Loss: 8.0240
Epoch 0, Treat Prop: 0.50, Loss: 3.2002
Epoch 50, Treat Prop: 0.02, Loss: 0.3704
Epoch 50, Treat Prop: 0.50, Loss: 0.9990
Epoch 100, Treat Prop: 0.02, Loss: 0.2022
Epoch 100, Treat Prop: 0.50, Loss: 0.5158
Epoch 150, Treat Prop: 0.02, Loss: 0.0390
Epoch 150, Treat Prop: 0.50, Loss: 0.0761
Epoch 200, Treat Prop: 0.02, Loss: 0.0229
Epoch 200, Treat Prop: 0.50, Loss: 0.0602
Epoch 250, Treat Prop: 0.02, Loss: 0.0207
Epoch 250, Treat Prop: 0.50, Loss: 0.0578
Epoch 300, Treat Prop: 0.02, Loss: 0.0200
Epoch 300, Treat Prop: 0.50, Loss: 0.0567
Epoch 350, Treat Prop: 0.02, Loss: 0.0198
Epoch 350, Treat Prop: 0.50, Loss: 0.0559


  0%|          | 1/1000 [00:05<1:27:29,  5.25s/it]

...

0.05438108742237091
Seed: 998
Epoch 0, Treat Prop: 0.02, Loss: 8.0210
Epoch 0, Treat Prop: 0.50, Loss: 3.2208
Epoch 50, Treat Prop: 0.02, Loss: 0.3635
Epoch 50, Treat Prop: 0.50, Loss: 0.9776
Epoch 100, Treat Prop: 0.02, Loss: 0.1839
Epoch 100, Treat Prop: 0.50, Loss: 0.4671
Epoch 150, Treat Prop: 0.02, Loss: 0.0349
Epoch 150, Treat Prop: 0.50, Loss: 0.0654
Epoch 200, Treat Prop: 0.02, Loss: 0.0234
Epoch 200, Treat Prop: 0.50, Loss: 0.0566
Epoch 250, Treat Prop: 0.02, Loss: 0.0216
Epoch 250, Treat Prop: 0.50, Loss: 0.0554
Epoch 300, Treat Prop: 0.02, Loss: 0.0210
Epoch 300, Treat Prop: 0.50, Loss: 0.0545
Epoch 350, Treat Prop: 0.02, Loss: 0.0207
Epoch 350, Treat Prop: 0.50, Loss: 0.0539


100%|█████████▉| 999/1000 [1:26:09<00:05,  5.45s/it]

0.05334242805838585
Seed: 999
Epoch 0, Treat Prop: 0.02, Loss: 8.0032
Epoch 0, Treat Prop: 0.50, Loss: 3.1539
Epoch 50, Treat Prop: 0.02, Loss: 0.3684
Epoch 50, Treat Prop: 0.50, Loss: 0.9686
Epoch 100, Treat Prop: 0.02, Loss: 0.2037
Epoch 100, Treat Prop: 0.50, Loss: 0.5046
Epoch 150, Treat Prop: 0.02, Loss: 0.0377
Epoch 150, Treat Prop: 0.50, Loss: 0.0754
Epoch 200, Treat Prop: 0.02, Loss: 0.0214
Epoch 200, Treat Prop: 0.50, Loss: 0.0571
Epoch 250, Treat Prop: 0.02, Loss: 0.0192
Epoch 250, Treat Prop: 0.50, Loss: 0.0556
Epoch 300, Treat Prop: 0.02, Loss: 0.0188
Epoch 300, Treat Prop: 0.50, Loss: 0.0550
Epoch 350, Treat Prop: 0.02, Loss: 0.0188
Epoch 350, Treat Prop: 0.50, Loss: 0.0546


100%|██████████| 1000/1000 [1:26:15<00:00,  5.18s/it]

0.05438044294714928
